In [1]:
from azureml.core import Workspace, Experiment


ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-130187
Azure region: southcentralus
Subscription id: f39cb977-6a3a-445b-a26a-b9a791c5fd89
Resource group: aml-quickstarts-130187


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
import pandas as pd

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = exp.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "cpu-cluster-4"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import ScriptRunConfig
import shutil
import os

# Specify parameter sampler
from azureml.train.hyperdrive import choice, loguniform
ps = RandomParameterSampling(
    {
        '--C': choice(0.2, 0,5, 1),
        '--max_iter': choice(25, 50, 100)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")


# Create a SKLearn estimator for use with train.py
shutil.copy('train.py', "./training")
est = SKLearn(
    source_directory= "./training",
    compute_target=compute_target,
    entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3593b936-7424-4ec2-bdc6-0ffb02cc0ccd
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_3593b936-7424-4ec2-bdc6-0ffb02cc0ccd?wsid=/subscriptions/f39cb977-6a3a-445b-a26a-b9a791c5fd89/resourcegroups/aml-quickstarts-130187/workspaces/quick-starts-ws-130187

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-12T08:55:03.692813][API][INFO]Experiment created<END>\n""<START>[2020-12-12T08:55:04.383225][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-12-12T08:55:04.694408][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-12T08:55:05.2337516Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_3593b936-7424-4ec2-bdc6-0ffb02cc0ccd
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_3593b936-7424-4ec2-bdc6-0ffb02cc0ccd?wsid=/subscriptions/f39cb

In [5]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
print(best_run)


['--C', '1', '--max_iter', '25']
['azureml-logs/55_azureml-execution-tvmps_48ad756841aa2111bdd5de169cb7570bec88fa9589928e13278375ff90aa3813_d.txt', 'azureml-logs/65_job_prep-tvmps_48ad756841aa2111bdd5de169cb7570bec88fa9589928e13278375ff90aa3813_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_48ad756841aa2111bdd5de169cb7570bec88fa9589928e13278375ff90aa3813_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/102_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']
Run(Experiment: udacity-project,
Id: HD_3593b936-7424-4ec2-bdc6-0ffb02cc0ccd_0,
Type: azureml.scriptrun,
Status: Completed)


In [6]:
model = best_run.register_model(model_name='first', model_path='outputs/model.joblib')

In [7]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'Regularization Strength:': 1.0,
 'Max iterations:': 25,
 'Accuracy': 0.7792207792207793}

In [8]:
print('Bets Run ID', best_run.id)

Bets Run ID HD_3593b936-7424-4ec2-bdc6-0ffb02cc0ccd_0
